In [48]:
#Instalando mesa con la librerías que se van a ocupar
!pip install numpy scipy matplotlib seaborn scikit-learn mesa==3.0 -q


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
fileref = open("ArchProj.txt", "r")
texto = fileref.readlines()
fileref.close()
grids = texto[:6]
victim = texto[6:15]
Fire = texto[15:24]
Door_lines = texto[24:37]
Access_point = texto[37:43]
print(grids)
print(victim)
print(Fire)
print(Door_lines)
print(Access_point)

['1011 1110 1011 1010 1000 1100 1001 1100\n', '1000 1000 1010 1110 0011 0110 0011 0110\n', '0011 0110 1101 1101 1111 1011 1110 1101\n', '1011 1010 0110 0111 1011 1010 1010 0110\n', '1001 1110 1001 1100 1101 1101 1101 1101\n', '0111 1011 0010 0110 0111 0111 0111 0111\n']
['1 1 f\n', '1 2 f\n', '1 8 v\n', '2 4 v\n', '3 5 v\n', '4 1 v\n', '6 2 v\n', '6 5 v\n', '6 7 v\n']
['1 3\n', '1 7\n', '2 3\n', '3 3\n', '5 2\n', '5 5\n', '5 7\n', '5 8\n', '6 3\n']
['1 2 1 3\n', '1 6 1 7\n', '2 4 2 5\n', '2 8 3 8\n', '3 3 3 4\n', '3 4 3 5\n', '3 5 3 6\n', '3 6 4 6\n', '4 4 5 4\n', '4 5 5 5\n', '4 6 5 6\n', '5 2 5 3\n', '6 6 6 7\n']
['1 4\n', '1 6\n', '2 1\n', '4 6\n', '6 4\n', '6 6\n']


In [50]:
#Conexion
import socket
import json
#Conexión
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import heapq, random
from os import read

In [51]:

# -------------------------------
# Leer txt y colocar en variables
# -------------------------------
fileref = open("ArchProj.txt", "r")
texto = fileref.readlines()
fileref.close()
grids = texto[:6]
victim = texto[6:15]
Fire = texto[15:24]
Door_lines = texto[24:37]
Access_point = texto[37:43]

# ------------------------------
# Parsear listas desde el archivo
# funciones realizada con ayuda de IA
# ------------------------------
def parse_positions(lines):
    coords = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 2:
            x, y = map(int, parts[:2])
            coords.append((y-1, x-1))  #(row, col)
    return coords

def parse_positions_v_f(lines):
    coords = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 3:
            y, x = map(int, parts[:2])  # OJO: primero fila (y), luego columna (x)
            kind = parts[2]
            coords.append((y-1, x-1, kind))
    return coords

def parse_doors(lines):
    coords = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 4:
            x1, y1, x2, y2 = map(int, parts[:4])
            coords.append((y1-1, x1-1, y2-1, x2-1))  # (row,col,row,col)
    return coords


fire_coords = parse_positions(Fire)
door_coords = parse_doors(Door_lines)
print("Puertas:", door_coords)
access_coords = parse_positions(Access_point)
print("Puntos de acceso:", access_coords)
false_alarm_victims_coords = parse_positions_v_f(victim)
print("Falsas alarmas y víctimas:", false_alarm_victims_coords)

Puertas: [(1, 0, 2, 0), (5, 0, 6, 0), (3, 1, 4, 1), (7, 1, 7, 2), (2, 2, 3, 2), (3, 2, 4, 2), (4, 2, 5, 2), (5, 2, 5, 3), (3, 3, 3, 4), (4, 3, 4, 4), (5, 3, 5, 4), (1, 4, 2, 4), (5, 5, 6, 5)]
Puntos de acceso: [(3, 0), (5, 0), (0, 1), (5, 3), (3, 5), (5, 5)]


In [52]:
# ------------------------------
# Valores para el mapa
# ------------------------------
EMPTY = 0
WALL = 1
VICTIM = 4
FALSE_ALARM = 5

###############################################################################

# ------------------------------
# Clase pared
# ------------------------------
class Cell:
    def __init__(self, up, left, down, right):
        self.positions = [up, left, down, right]
        self.damages = [2,2,2,2]

    def get_damages(self, side = None):
        if side == None:
            return self.damages
        else:
            return self.damages[side]

    def get_positions(self, side = None):
        if side != None:
            return self.positions[side]
        else:
            return self.positions

    def set_damage(self, side, damage):
        if self.damages[side] <= 0:
            return

        self.damages[side] -= damage
        if self.damages[side] <= 0:
            print(f"Pared destruida en lado {side}")

    def is_empty(self):
      for position in self.positions:
        if position != EMPTY:
          return False
        else:
          return True

    def set_cell(self,values):
        if len(values) == 4:
            self.positions = values
        return self.positions

# ------------------------------
# Crear matriz de celdas 6x8 a
# partir de `grids`
# Función creada con ayuda de IA
# ------------------------------
height, width = 6, 8
matriz_pared = [[None for _ in range(width)] for _ in range(height)]

for i in range(height): # recorre las 6 fila
  fila = grids[i].strip().split() # separa en bloques (ej: '1011', '1110')
  for j in range(width): # recorre las 8 columnas
    valores = [int(x) for x in fila[j]] # convierte '1011' → [1,0,1,1]
    matriz_pared[i][j] = Cell(*valores) # desempaqueta en up,left,down,righ

for i in range(height):
  for j in range(width):
    print(matriz_pared[i][j].get_positions(), end=" ")
  print()

  for x,y in access_coords:
    matriz_pared[y][x].set_cell([0,0,0,0]) # acceso libre

###############################################################################
# ------------------------------
# Clase Puertas
# ------------------------------
# 0 cerrada
# 1 abierta
# 2 destruida
class Door:
    CLOSED = 0
    OPEN = 1
    DESTROYED = 2

    def __init__(self, r1, c1, r2, c2):
        self.state = Door.CLOSED
        self.cell1 = (r1, c1)
        self.cell2 = (r2, c2)

    def get_state(self):
        return self.state

    def set_state(self, state):
        self.state = state

    def open(self):
        if self.state == Door.CLOSED:
            self.state = Door.OPEN

    def close(self):
        if self.state == Door.OPEN:
            self.state = Door.CLOSED

    def destroy(self):
        self.state = Door.DESTROYED

    def connects(self, cell):
        if cell == self.cell1:
            return self.cell2
        elif cell == self.cell2:
            return self.cell1
        return None

    def blocks_passage(self):
        return self.state == Door.CLOSED


###############################################################################
door_list = []
for (r1, c1, r2, c2) in door_coords:
    door = Door(r1, c1, r2, c2)
    door_list.append(door)

cell_doors = {}

for door in door_list:
    for cell in [door.cell1, door.cell2]:
        if cell not in cell_doors:
            cell_doors[cell] = []
        cell_doors[cell].append(door)

"""
print("Puertas por celda:")
for cell, doors in cell_doors.items():
    print(f"Celda {cell}: {len(doors)} puertas")
"""
###############################################################################
#----------------------
# clase fire
#----------------------
class Fire:
    EMPTY = 0
    SMOKE = 1
    FIRE = 2
    EXPLODED = 3

    def __init__(self, row, col):
        self.state = Fire.EMPTY
        self.row = row
        self.col = col
        self.to_explode = False

    def get_state(self):
        return self.state

    def set_state(self, state):
        self.state = state

    def get_position(self):
        return (self.row, self.col)

    def get_neighbors(self, matriz_pared):
        neighbors = []
        # arriba, derecha, abajo, izquierda
        directions = [(-1,0,0), (0,1,1), (1,0,2), (0,-1,3)]
        for dr, dc, side in directions:
            nr, nc = self.row + dr, self.col + dc
            if 0 <= nr < len(matriz_pared) and 0 <= nc < len(matriz_pared[0]):
                if matriz_pared[self.row][self.col].get_positions(side) == 0 and \
                   matriz_pared[nr][nc].get_positions((side + 2) % 4) == 0:
                    neighbors.append((nr, nc, side))
        return neighbors

    def advance_fire(self):
        r = random.randint(0, len(self.fire_grid)-1)
        c = random.randint(0, len(self.fire_grid[0])-1)
        target = self.fire_grid[r][c]

        if target.state == Fire.EMPTY:
            target.state = Fire.SMOKE
        elif target.state == Fire.SMOKE:
            target.state = Fire.FIRE
        elif target.state == Fire.FIRE:
            target.to_explode = True


    def propagate_explosion(self, matriz_fire, matriz_pared, model=None):
        # Explosión en 4 direcciones
        for nr, nc, side in self.get_neighbors(matriz_pared):
            vecino = matriz_fire[nr][nc]
            if vecino.state == Fire.EMPTY:
                vecino.state = Fire.SMOKE
            elif vecino.state == Fire.SMOKE:
                vecino.state = Fire.FIRE
            
            # Daño a paredes
            if matriz_pared[nr][nc].get_positions(side) == 1:
                matriz_pared[nr][nc].set_damage(side, 1)
                model.total_damage += 1
                print(f"Daño en pared por fuego en: {(nr, nc)} lado {side}")

        self.to_explode = False

    def set_smoke(self, matriz_fire, matriz_pared):
        # Paso de flashover: humo adyacente a fuego
        for nr, nc, _ in self.get_neighbors(matriz_pared):
            if matriz_fire[nr][nc].state == Fire.SMOKE:
                matriz_fire[nr][nc].state = Fire.FIRE

###############################################################################
print(fire_coords)
height, width = 6, 8
matriz_fire = [[Fire(i,j) for j in range(width)] for i in range(height)]

for (x,y) in fire_coords:
  matriz_fire[y][x].set_state(2)

for i in range(height):
    for j in range(width):
        print(matriz_fire[i][j].get_state(), end=" ")
    print()
"""
aux = 0
while aux < 10:
    print(f"Paso {aux}")
    for i in range(height):
        for j in range(width):
            print(matriz_fire[i][j].get_state(), end=" ")
        print()
    print()
    # Expandir fuegos
    for i in range(height):
        for j in range(width):
            if matriz_fire[i][j].get_state() == Fire.FIRE:
                matriz_fire[i][j].expand(matriz_fire, matriz_pared)
    # Propagar explosiones
    for i in range(height):
        for j in range(width):
            if matriz_fire[i][j].to_explode:
                matriz_fire[i][j].propagate_explosion(matriz_fire, matriz_pared)
    aux += 1
"""

###############################################################################
# Clase Victim or False Alarm
###############################################################################
class POI:
    VICTIM = "v"
    FALSE = "f"

    def __init__(self, row, col, role):
        self.row = row
        self.col = col
        self.role = role
        self.revealed = False

    def reveal(self):
        self.revealed = True

    def is_victim(self):
        return self.role == POI.VICTIM and self.revealed

    def is_false(self):
        return self.role == POI.FALSE and self.revealed

    def get_position(self):
        return (self.row, self.col)


###############################################################################
# Corrección de matriz de POIs
###############################################################################

matriz_POI = [[None for j in range(width)] for i in range(height)]
for (row, col, role) in false_alarm_victims_coords:   # fila, col, rol
        matriz_POI[row][col] = POI(row, col, role)

"""
print("Matriz de POIs:")
for i in range(height):
    for j in range(width):
        if matriz_POI[i][j] is None:
            print(" . ", end=" ")
        else:
            print(f" {matriz_POI[i][j].role} ", end=" ")
    print()
"""

[1, 0, 1, 1] [1, 1, 1, 0] [1, 0, 1, 1] [1, 0, 1, 0] [1, 0, 0, 0] [1, 1, 0, 0] [1, 0, 0, 1] [1, 1, 0, 0] 
[0, 0, 0, 0] [1, 0, 0, 0] [1, 0, 1, 0] [1, 1, 1, 0] [0, 0, 1, 1]

 [0, 1, 1, 0] [0, 0, 1, 1] [0, 1, 1, 0] 
[0, 0, 1, 1] [0, 1, 1, 0] [1, 1, 0, 1] [1, 1, 0, 1] [1, 1, 1, 1] [1, 0, 1, 1] [1, 1, 1, 0] [1, 1, 0, 1] 
[1, 0, 1, 1] [1, 0, 1, 0] [0, 1, 1, 0] [0, 1, 1, 1] [1, 0, 1, 1] [0, 0, 0, 0] [1, 0, 1, 0] [0, 1, 1, 0] 
[1, 0, 0, 1] [1, 1, 1, 0] [1, 0, 0, 1] [1, 1, 0, 0] [1, 1, 0, 1] [1, 1, 0, 1] [1, 1, 0, 1] [1, 1, 0, 1] 
[0, 1, 1, 1] [1, 0, 1, 1] [0, 0, 1, 0] [0, 0, 0, 0] [0, 1, 1, 1] [0, 0, 0, 0] [0, 1, 1, 1] [0, 1, 1, 1] 
[(2, 0), (6, 0), (2, 1), (2, 2), (1, 4), (4, 4), (6, 4), (7, 4), (2, 5)]
0 0 2 0 0 0 2 0 
0 0 2 0 0 0 0 0 
0 0 2 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 2 0 0 2 0 2 2 
0 0 2 0 0 0 0 0 


'\nprint("Matriz de POIs:")\nfor i in range(height):\n    for j in range(width):\n        if matriz_POI[i][j] is None:\n            print(" . ", end=" ")\n        else:\n            print(f" {matriz_POI[i][j].role} ", end=" ")\n    print()\n'

In [53]:
# ------------------------------
# Diccionario de direcciones (fila, col, lado)
# ------------------------------
DIRECTIONS = {
    "up":    (-1, 0, 0),  # fila-1, col, lado 0
    "left":  (0, -1, 1),  # fila, col-1, lado 1
    "down":  (1, 0, 2),   # fila+1, col, lado 2
    "right": (0, 1, 3)    # fila, col+1, lado 3
}

# ------------------------------
# Agente con AP
# ------------------------------
class RanaAgent(Agent):
    def __init__(self, model):
        super().__init__(model)
        self.Initial_AP = 4
        self.AP = self.Initial_AP
        self.carry = False

    def sum_AP(self):
        """Recargar AP al inicio del turno (máx. 8)"""
        self.AP = min(self.AP + self.Initial_AP, 8)
        return self.AP

    def can_move(self, direction, matriz_pared):
        col, row = self.pos   # (x, y)
        drow, dcol, side = DIRECTIONS[direction]
        nrow, ncol = row + drow, col + dcol

        # límites contra la matriz (fila, col)
        if not (0 <= nrow < len(matriz_pared) and 0 <= ncol < len(matriz_pared[0])):
            return False

        current_cell  = matriz_pared[row][col]
        neighbor_cell = matriz_pared[nrow][ncol]

        if current_cell.get_positions(side) == 1:
            return False
        if neighbor_cell.get_positions((side + 2) % 4) == 1:
            return False

        cell1 = (row, col)
        cell2 = (nrow, ncol)
        for door in cell_doors.get(cell1, []):
            if door.connects(cell1) == cell2 and door.blocks_passage():
                return False

        return True



    def get_valid_moves(self, matriz_pared):
        """Devuelve todos los movimientos válidos desde la celda actual"""
        valid_moves = []
        # OJO: self.pos es (x, y) = (col, row)
        col, row = self.pos
        for direction in DIRECTIONS:
            if self.can_move(direction, matriz_pared):
                drow, dcol, side = DIRECTIONS[direction]
                nrow = row + drow
                ncol = col + dcol
                valid_moves.append((nrow, ncol, side))
        return valid_moves


    def move(self, matriz_pared, matriz_fire, matriz_POI):
        moves = self.get_valid_moves(matriz_pared)
        if not moves or self.AP <= 0:
            return

        # 🔹 Elegir nueva posición
        nrow, ncol, side = random.choice(moves)

        # 🔹 Mover al agente en el grid de Mesa (x=col, y=fila)
        self.model.grid.move_agent(self, (ncol, nrow))
        self.AP -= 1

        # 🔹 Apagar fuego
        fire_cell = matriz_fire[nrow][ncol]
        if fire_cell.get_state() == Fire.FIRE and self.AP >= 2:
            fire_cell.set_state(Fire.EMPTY)
            self.AP -= 2

        # 🔹 Destruir pared
        cell = matriz_pared[nrow][ncol]
        if cell.get_positions(side) == 1 and self.AP >= 2:
            if random.random() < 0.2:
                cell.set_damage(side, 1)
                self.model.total_damage += 1
                print("Daño total del edificio por rana:", self.model.total_damage)
                self.AP -= 2

        # 🔹 Revisar POI
        poi = self.model.poi_grid[nrow][ncol]
        if poi and not poi.revealed:
            poi.reveal()
            if poi.is_victim() and not self.carry:
                self.carry = True

        # 🔹 Entregar víctima en salida
        if self.carry and (nrow, ncol) in access_coords:
            self.carry = False
            self.model.rescued += 1
            print("Víctima entregada en salida")

         # 🔹 Recargar AP
        self.sum_AP()
        print(f"Rana en {self.pos} con AP={self.AP} y carry={self.carry}")

    def step(self):
        self.move(matriz_pared, matriz_fire, matriz_POI)


# ------------------------------
# Modelo
# ------------------------------
class RanaModel(Model):
    def __init__(self, width=8, height=6, agents=6):
        super().__init__()
        self.grid = MultiGrid(width, height, torus=False)
        self.schedule = RandomActivation(self)

        self.total_damage = 0
        self.max_damage = 24
        self.rescued = 0
        self.lost = 0
        self.rescue_goal = 7
        self.lose_limit = 4
        self.collapsed = False

        self.datacollector = DataCollector(
    model_reporters={
        "Grid": get_grid,
        "Step": lambda m: m.schedule.steps,   # número de pasos
        "Rescued": lambda m: m.rescued,      # víctimas rescatadas
        "Lost": lambda m: m.lost,            # víctimas perdidas
        "TotalDamage": lambda m: m.total_damage,  # daño total
        "Collapsed": lambda m: m.collapsed,  # colapso (True/False)
    }
)

        for (row, col) in access_coords:  # acceder con fila, col
            print("Abriendo puerta en:", (row, col))
            agent = RanaAgent(self)
            if 0 <= row < height and 0 <= col < width:
                self.grid.place_agent(agent, (col, row))
                self.schedule.add(agent)
        
        self.wall_grid = matriz_pared
        self.fire_grid = matriz_fire
        self.poi_grid = matriz_POI
        print("Agentes creados:", len(self.schedule.agents))

    # Nuevo método para simular la tirada de dados y colocar humo/fuego
    def advance_fire(self):
        r = random.randint(0, len(self.fire_grid)-1)
        c = random.randint(0, len(self.fire_grid[0])-1)
        target = self.fire_grid[r][c]

        if target.state == Fire.EMPTY:
            target.state = Fire.SMOKE
        elif target.state == Fire.SMOKE:
            target.state = Fire.FIRE
        elif target.state == Fire.FIRE:
            target.to_explode = True

    def step(self):
        if self.collapsed:
            return

        self.schedule.step()

        # Revisar colapso del edificio
        if self.total_damage >= self.max_damage and not self.collapsed:
            print("DERROTA: edificio colapsado")
            self.collapsed = True
            return

        # Revisar victoria
        if self.rescued >= self.rescue_goal:
            print("VICTORIA: suficientes víctimas rescatadas")
            return

        # Recolectar datos
        self.datacollector.collect(self)

        # 1. Avance del fuego
        self.advance_fire()

        # 2. Propagar explosiones
        for row in self.fire_grid:
            for cell in row:
                if cell.to_explode:
                    cell.propagate_explosion(self.fire_grid, matriz_pared, self)

        # 3. Flashover
        for row in self.fire_grid:
            for cell in row:
                if cell.state == Fire.FIRE:
                    cell.set_smoke(self.fire_grid, matriz_pared)

        # Revisar víctimas alcanzadas por fuego
        for i in range(len(self.fire_grid)):
            for j in range(len(self.fire_grid[0])):
                poi = self.poi_grid[i][j]
                if self.fire_grid[i][j].state == Fire.FIRE and poi and poi.role == POI.VICTIM:
                    self.lost += 1
                    self.poi_grid[i][j] = None
                    if self.lost >= self.lose_limit:
                        print("DERROTA: demasiadas víctimas perdidas")
                        return


def get_grid(model):
    """Convierte el grid de Mesa en una matriz numpy para análisis o visualización"""
    grid = np.zeros((model.grid.height, model.grid.width))
    for (content, (x, y)) in model.grid.coord_iter():
        if len(content) > 0:
            grid[y][x] = 1   # usar y como fila, x como columna
    return grid



In [54]:
# ------------------------------
# Ejecutar y Animar
# ------------------------------
width=8
height=6
agents=6
model = RanaModel(width, height, agents)
for i in range(30):
  print(f"Paso {i}")
  model.step()
  if model.collapsed or model.rescued >= model.rescue_goal or model.lost >= model.lose_limit:
    break


C:\Users\mbajo\AppData\Local\Temp\ipykernel_14684\2299106134.py:117: DeprecationWarning: The time module and all its Schedulers are deprecated and will be removed in a future version. They can be replaced with AgentSet functionality. See the migration guide for details. https://mesa.readthedocs.io/latest/migration_guide.html#time-and-schedulers
  self.schedule = RandomActivation(self)


Abriendo puerta en: (3, 0)
Abriendo puerta en: (5, 0)


Abriendo puerta en: (0, 1)
Abriendo puerta en: (5, 3)
Abriendo puerta en: (3, 5)
Abriendo puerta en: (5, 5)
Agentes creados: 6
Paso 0
Rana en (6, 3) con AP=7 y carry=False
Rana en (2, 0) con AP=5 y carry=False
Rana en (5, 4) con AP=7 y carry=False
Rana en (0, 4) con AP=7 y carry=False
Rana en (3, 4) con AP=7 y carry=False
Paso 1
Rana en (1, 0) con AP=8 y carry=False
Rana en (5, 5) con AP=8 y carry=False
Daño total del edificio por rana: 1
Rana en (5, 0) con AP=6 y carry=False
Daño total del edificio por rana: 2
Rana en (0, 1) con AP=4 y carry=False
Rana en (3, 5) con AP=8 y carry=False
Daño total del edificio por rana: 3
Rana en (5, 5) con AP=6 y carry=False
Daño total del edificio por rana: 4
Rana en (5, 3) con AP=6 y carry=False
Paso 1
Daño total del edificio por rana: 5
Rana en (5, 3) con AP=8 y carry=False
Daño total del edificio por rana: 6
Rana en (0, 1) con AP=6 y carry=False
Rana en (3, 5) con AP=8 y carry=False
Daño total del edificio por rana: 7
Rana en (5, 0) con AP=8 y carr

In [43]:
# Datos a nivel de modelo
df_model = model.datacollector.get_model_vars_dataframe()
print(df_model)

                                                 Grid  Step  Rescued  Lost  \
0   [[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....     1        0     0   
1   [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....     2        0     0   
2   [[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....     3        0     0   
3   [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....     4        0     0   
4   [[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....     5        0     0   
5   [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....     6        0     0   
6   [[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....     7        0     1   
7   [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....     8        0     1   
8   [[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....     9        0     1   
9   [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....    10        0     1   
10  [[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....    11        0     1   
11  [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0....    12     

In [ ]:
#Conexion local a C#
#Código creado cguiado mediante IA

HOST = "127.0.0.1"
PORT = 65432

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((HOST, PORT))
server.listen()
print(f"Servidor escuchando en {HOST}:{PORT}...")

conn, addr = server.accept()
print(f"Cliente conectado desde {addr}")

ITERATIONS = 30
model = RanaModel()


try:
    for i in range(ITERATIONS):
        model.step()
        grid = get_grid(model).tolist()   #convertir numpy → lista

        data = {
            #Funciones 
            "step": i,
            "grid": grid, 
            #Importar agente
            "agent_pos": ([agent.pos for agent in model.schedule.agents]),
            "agent_ap": ([agent.AP for agent in model.schedule.agents]),
            "agent_carry": ([agent.carry for agent in model.schedule.agents]),
            #Importar fuego
            "fires": ([[cell.row, cell.col, cell.state] for row in model.fire_grid for cell in row if cell.state != Fire.EMPTY]),
            "fire_grid": ([[cell.row, cell.col, cell.state] for row in model.fire_grid for cell in row]),
            #Importar POI
            "pois": ([[poi.row, poi.col, poi.role, poi.revealed] for row in model.poi_grid for poi in row if poi is not None]),
            #Importar paredes
            "wall_grid": ([[cell.get_positions() for cell in row] for row in model.wall_grid]),
            "wall_damages": ([[cell.get_damages() for cell in row] for row in model.wall_grid]),
            #Importar puertas
            "door_list": ([[door.cell1, door.cell2, door.state] for door in door_list]),
            #Modelo
            "model.collapsed": model.collapsed,
            "total_damage": model.total_damage,
            "rescued": model.rescued,
            "lost": model.lost,
            "max_damage": model.max_damage,
        }


        message = json.dumps(data)
        conn.sendall((message + "\n").encode("utf-8"))
        print(f"Enviado paso {i}")

finally:
    conn.close()
    server.close()

Servidor escuchando en 127.0.0.1:65432...


In [ ]:
anim